<a href="https://colab.research.google.com/github/rasecfaria/FinalProject/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Sistema de Recomendação`🎈

In [3]:
#importar bibliotecas
import pandas as pd
import numpy as np

### **1.Analise e manipulação dos dados**

In [4]:
#import dos filmes e dos ratings
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
#ratings.head(10)
#movies.head(10)

In [5]:
#remover o timesstamp dos ratings(provisorio, check notion)
ratings = ratings [['userId', 'movieId', 'rating']]
#ratings.head(10)

In [7]:
#Só dá output a uma coisa de cada vez por isso tem que ser um de cada vez ou colocar em nova linha
ratings.shape
#movies.shape

(100836, 3)

In [8]:
#numero de avaliacoes de cada utilizador
#ratings['userId'].value_counts()

In [9]:
#junta tudo num unico  dataframe, a partir do movieId que é fator comum nos ratings e movies
df = movies.merge(ratings, on='movieId')
#df.head()

# ***1º metodo -> filtragem colaborativa***

In [ ]:
movies_table = df.pivot_table(index='title', columns='userId', values='rating').fillna(0)
movies_table.head(5)

In [35]:
import sklearn.metrics.pairwise as pw

In [ ]:
#isto indica a similiariedade entre cada filme, logicamente filmes iguais é 1 e as proximidades quanto mais proximo for de 1
#isto é feito atraves do cosseno entre os vetores dos ratings de cada filme.
rec = pw.cosine_similarity(movies_table)
rec_df = pd.DataFrame(rec, columns=movies_table.index, index=movies_table.index)
rec_df.head(5)

In [ ]:
cossine_df = pd.DataFrame(rec_df['Forrest Gump'].sort_values(ascending=False))
cossine_df.columns = ['recomendações']
cossine_df.head(10)

# ***2º metodo -> baseado em conteudo***

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
tags = pd.read_csv('tags.csv')
dados = pd.read_csv('dados.csv')

In [82]:
tags.head()
#dados.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


In [83]:
tags.shape
#dados.shape

(45256, 2)

In [84]:
movies['movieId'] =  movies['movieId'].apply(lambda x: str(x))

In [85]:
df2 = movies.merge(dados, left_on='title', right_on='Name', how='left')
df2 = df2.merge(tags, left_on='movieId', right_on='movieId', how='left')
df2['Infos'] = df2['genres'] #+ str(df2['Discription']) + df2['tag']
df2.head()

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,Infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",Comedy


In [86]:
vec  = TfidfVectorizer()
tfidf = vec.fit_transform(df2['Infos'].apply(lambda x: np.str_(x)))

In [87]:
sim = cosine_similarity(tfidf)
sim


array([[1.        , 0.81140572, 0.15520332, ..., 0.        , 0.42336767,
        0.27023383],
       [0.81140572, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15520332, 0.        , 1.        , ..., 0.        , 0.        ,
        0.57432973],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.42336767, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.27023383, 0.        , 0.57432973, ..., 0.        , 0.        ,
        1.        ]])

In [88]:
sim_df2 = pd.DataFrame(sim, columns=df2['title'], index=df2['title'])
sim_df2.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Heat,Sabrina,Tom and Huck,Sudden Death,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.811406,0.155203,0.137322,0.270234,0.0,0.0,0.155203,0.654704,0.0,...,0.362939,0.467560,0.199169,0.518261,0.0,0.680357,0.755885,0.000000,0.423368,0.270234
Jumanji,0.811406,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.806877,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.337153,0.374581,0.000000,0.000000,0.000000
Grumpier Old Men,0.155203,0.000000,1.000000,0.884789,0.574330,0.0,0.0,1.000000,0.000000,0.0,...,0.164889,0.000000,0.423296,0.000000,0.0,0.184810,0.205327,0.000000,0.000000,0.574330
Waiting to Exhale,0.137322,0.000000,0.884789,1.000000,0.508161,0.0,0.0,0.884789,0.000000,0.0,...,0.145892,0.201018,0.689475,0.000000,0.0,0.163518,0.181671,0.465991,0.000000,0.508161
Father of the Bride Part II,0.270234,0.000000,0.574330,0.508161,1.000000,0.0,0.0,0.574330,0.000000,0.0,...,0.287098,0.000000,0.737026,0.000000,0.0,0.321784,0.357506,0.000000,0.000000,1.000000


In [89]:
final_df = pd.DataFrame(sim_df2['Forrest Gump'].sort_values(ascending=False))
final_df.columns = ['recomendações']
final_df.head(10)

,recomendações
title,
I Served the King of England (Obsluhoval jsem anglického krále),1.00000
Train of Life (Train de vie),1.00000
Forrest Gump,1.00000
"Tiger and the Snow, The (La tigre e la neve)",1.00000
Life Is Beautiful (La Vita è bella),1.00000
Gone with the Wind,0.93973
Malèna,0.93973
"Sand Pebbles, The",0.93973
Cold Mountain,0.93973


# **Tutorial 1 - basico**

In [63]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9737 entries, 0 to 9736
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9737 non-null   object
 1   title    9735 non-null   object
 2   genres   9737 non-null   object
 3   year     9737 non-null   object
dtypes: object(4)
memory usage: 304.4+ KB


In [64]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [65]:
#juncao dos dataframes
df = ratings.merge(movies, on='movieId')
df.head(10)

ValueError: You are trying to merge on int64 and object columns for key 'movieId'. If you wish to proceed you should use pd.concat

In [ ]:
#pivot quero que cada userID seja uma variave com o respetivo valor de nota para cada filme
movie_pivot = df.pivot_table(columns='userId', values='rating', index='title')
movie_pivot.head(10)

In [ ]:
#preencher valores nulos com 0
movie_pivot.fillna(0, inplace=True)

In [ ]:
#criar matrix sparsa
from scipy.sparse import csr_matrix
movie_sparse = csr_matrix(movie_pivot)

In [ ]:
#importar o algoritmo KNN do Scikit
from sklearn.neighbors import NearestNeighbors

In [ ]:
#treinar o modelo
model = NearestNeighbors(algorithm='brute')
model.fit(movie_sparse)

Previsao de sugestao de filmes

In [ ]:
#Toy Story
distances, suggestions = model.kneighbors(movie_pivot.filter(items = ['Toy Story (1995)'], axis=0).values.reshape(1, -1))

for i in range(len(suggestions)):
  print(movie_pivot.index[suggestions[i]])

In [ ]:
#velocidade furiosa
distances, suggestions = model.kneighbors(movie_pivot.filter(items = ['Fast and the Furious, The (2001)'], axis=0).values.reshape(1, -1))

for i in range(len(suggestions)):
  print(movie_pivot.index[suggestions[i]])